In [14]:
from pyswarm import pso
import pandas as pd
import numpy as np
import random
import os
import random
import statistics as sts
import math
from imblearn.pipeline import make_pipeline
from sklearn.ensemble import ExtraTreesClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from utils import *
import warnings
warnings.filterwarnings("ignore")

In [15]:
from sklearn.feature_selection import VarianceThreshold
threshold = (.95 * (1 - .95))

In [16]:
from sklearn.preprocessing import MinMaxScaler

In [17]:
normalized_weight_list = []

lgb_mf1024_test_auc_list = []
xgb_mol2vec_test_auc_list = []
et_rd_test_auc_list = []
cat_padel_test_auc_list = []
meta_test_auc_list = []

lgb_mf1024_pred_list = []
xgb_mol2vec_pred_list = []
et_rd_pred_list = []
cat_padel_pred_list = []
meta_pred_list = []

In [18]:
train_rd      = pd.read_csv('rdkit_train.csv')
train_mf1024  = pd.read_csv('fp1024_train.csv')
train_padel  = pd.read_csv('padel_train.csv')
train_mol2vec = pd.read_csv('mol2vec_train.csv')
train_label   = pd.read_csv('label_train.csv')
    #--------------------------------------------------------------------#
    # Load validation data
val_rd      = pd.read_csv('rdkit_valid.csv')
val_mf1024  = pd.read_csv('fp1024_valid.csv')
val_padel  = pd.read_csv('padel_valid.csv')
val_mol2vec = pd.read_csv('mol2vec_valid.csv')
val_label   = pd.read_csv('label_valid.csv')
    #--------------------------------------------------------------------#
    # Load test data
test_rd      = pd.read_csv('rdkit_test.csv')
test_mf1024  = pd.read_csv('fp1024_test.csv')
test_padel  = pd.read_csv('padel_test.csv')
test_mol2vec = pd.read_csv('mol2vec_test.csv')
test_label   =pd.read_csv('label_test.csv')

In [19]:
train_rd.replace(to_replace='#NAME?',value=np.nan,inplace=True)
train_rd.replace(to_replace=r'^\s*$',value=np.nan,regex=True,inplace=True)
columns = train_rd.columns
train_rd = train_rd.astype("float", errors = "ignore")
train_rd = np.float32(train_rd)
train_rd[np.isnan(train_rd)] = 0
train_rd[np.isinf(train_rd)] = 0
train_rd=pd.DataFrame(train_rd,columns=columns)

In [20]:
val_rd.replace(to_replace='#NAME?',value=np.nan,inplace=True)
val_rd.replace(to_replace=r'^\s*$',value=np.nan,regex=True,inplace=True)
columns = val_rd.columns
val_rd = val_rd.astype("float", errors = "ignore")
val_rd = np.float32(val_rd)
val_rd[np.isnan(val_rd)] = 0
val_rd[np.isinf(val_rd)] = 0
val_rd=pd.DataFrame(val_rd,columns=columns)

In [21]:
test_rd.replace(to_replace='#NAME?',value=np.nan,inplace=True)
test_rd.replace(to_replace=r'^\s*$',value=np.nan,regex=True,inplace=True)
columns = test_rd.columns
test_rd = test_rd.astype("float", errors = "ignore")
test_rd = np.float32(test_rd)
test_rd[np.isnan(test_rd)] = 0
test_rd[np.isinf(test_rd)] = 0
test_rd=pd.DataFrame(test_rd,columns=columns)

In [22]:
train_padel.replace(to_replace='#NAME?',value=np.nan,inplace=True)
train_padel.replace(to_replace=r'^\s*$',value=np.nan,regex=True,inplace=True)
columns = train_padel.columns
train_padel = train_padel.astype("float", errors = "ignore")
train_padel = np.float32(train_padel)
train_padel[np.isnan(train_padel)] = 0
train_padel[np.isinf(train_padel)] = 0
train_padel=pd.DataFrame(train_padel,columns=columns)

In [23]:
val_padel.replace(to_replace='#NAME?',value=np.nan,inplace=True)
val_padel.replace(to_replace=r'^\s*$',value=np.nan,regex=True,inplace=True)
columns = val_padel.columns
val_padel = val_padel.astype("float", errors = "ignore")
val_padel = np.float32(val_padel)
val_padel[np.isnan(val_padel)] = 0
val_padel[np.isinf(val_padel)] = 0
val_padel=pd.DataFrame(val_padel,columns=columns)

In [24]:
test_padel.replace(to_replace='#NAME?',value=np.nan,inplace=True)
test_padel.replace(to_replace=r'^\s*$',value=np.nan,regex=True,inplace=True)
columns = test_padel.columns
test_padel = test_padel.astype("float", errors = "ignore")
test_padel = np.float32(test_padel)
test_padel[np.isnan(test_padel)] = 0
test_padel[np.isinf(test_padel)] = 0
test_padel=pd.DataFrame(test_padel,columns=columns)

In [26]:
# Create classifiers
my_lgb_mf1024 = make_pipeline(VarianceThreshold(),
                                LGBMClassifier(bagging_fraction=0.4, bagging_freq=3, boosting_type='gbdt',
               class_weight=None, colsample_bytree=1.0,
               feature_fraction=0.8316682559759652, importance_type='split',
               learning_rate=0.008022065621873459, max_depth=-1,
               min_child_samples=1, min_child_weight=0.001,
               min_split_gain=0.78642128858054, n_estimators=300, n_jobs=16,
               num_leaves=67, objective=None, random_state=123,
               reg_alpha=3.638943389135039e-08,
               reg_lambda=1.2845851602217718e-05, silent='warn', subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0))
my_xgb_mol2vec = make_pipeline(VarianceThreshold(),
                                XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5653409252688301, gamma=0,
              gpu_id=-1, importance_type='gain', interaction_constraints='',
              learning_rate=0.027093121856676457, max_delta_step=0, max_depth=8,
              min_child_weight=2, monotone_constraints='()',
              n_estimators=153, n_jobs=16, num_parallel_tree=1,
              objective='binary:logistic', random_state=123,
              reg_alpha=5.617540699056763e-06, reg_lambda=4.499201478224156e-09,
              scale_pos_weight=1.0, subsample=0.5745516271353704,
              tree_method='auto', use_label_encoder=True, validate_parameters=1,
              verbosity=0))
my_et_rd = make_pipeline(VarianceThreshold(),
                            ExtraTreesClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                     criterion='entropy', max_depth=9, max_features=0.4,
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=6.662855879168866e-07,
                     min_impurity_split=None, min_samples_leaf=1,
                     min_samples_split=2, min_weight_fraction_leaf=0.0,
                     n_estimators=209, n_jobs=16, oob_score=False,
                     random_state=123, verbose=0, warm_start=False))      
my_cat_padel = make_pipeline(VarianceThreshold(),
                                   CatBoostClassifier(nan_mode='Min', eval_metric='Logloss', iterations=300, 
                                                  sampling_frequency='PerTree', leaf_estimation_method='Newton', 
                                                  grow_policy='SymmetricTree', penalties_coefficient=1, boosting_type='Plain', 
                                                  model_shrink_mode='Constant', feature_border_type='GreedyLogSum', 
                                                  l2_leaf_reg=1, 
                                                  random_strength=0.800000011920929, rsm=1, boost_from_average=False, 
                                                  model_size_reg=0.5, subsample=0.800000011920929, 
                                                  use_best_model=False, class_names=[0, 1], random_seed=123, depth=6, 
                                                  posterior_sampling=False, border_count=254, classes_count=0, 
                                                  auto_class_weights=None, sparse_features_conflict_fraction=0, 
                                                  leaf_estimation_backtracking='AnyImprovement', best_model_min_trees=1, 
                                                  model_shrink_rate=0, min_data_in_leaf=1, loss_function='Logloss', 
                                                  learning_rate=0.058222733438014984, score_function='Cosine', task_type='CPU', 
                                                  leaf_estimation_iterations=10, bootstrap_type='MVS', max_leaves=64))
    #--------------------------------------------------------------------#
    # Get predicted probabilities of validation sets
val_pred_lgb_mf1024  = my_lgb_mf1024.fit(train_mf1024, train_label).predict_proba(val_mf1024)[::,1]
val_pred_xgb_mol2vec  = my_xgb_mol2vec.fit(train_mol2vec, train_label).predict_proba(val_mol2vec)[::,1]
val_pred_et_rd       = my_et_rd.fit(train_rd, train_label).predict_proba(val_rd)[::,1]
val_pred_cat_padel = my_cat_padel.fit(train_padel, train_label).predict_proba(val_padel)[::,1]
    #--------------------------------------------------------------------#
val_preds = [val_pred_lgb_mf1024, val_pred_xgb_mol2vec, val_pred_et_rd, val_pred_cat_padel]
auc_optimasation = get_optimasation_function(val_preds, val_label)
    #--------------------------------------------------------------------#
    # Define parameters for optimization
lb, ub = [0.1, 0.1, 0.1, 0.1], [0.5, 0.5, 0.5, 0.5]
weight_list = []
seed_range = np.arange(0,10)
    # Optimization loop
for s in seed_range:
    np.random.seed(s)
    random.seed(s)
    optx, fopt = pso(auc_optimasation, lb, ub, swarmsize=100, seed=s, maxiter=50)
    weight_list.append(extract_weight(optx))
    print("Round {}: Completed".format(s+1))
    # Extract raw weights
    w1_list, w2_list, w3_list, w4_list = [], [], [], []
    for weight in weight_list:
        w1_list.append(weight[0])
        w2_list.append(weight[1])
        w3_list.append(weight[2])
        w4_list.append(weight[3])
    # Normalize weights
    w1_avage_norm = np.round(np.mean(w1_list), 4)
    w2_avage_norm = np.round(np.mean(w2_list), 4)
    w3_avage_norm = np.round(np.mean(w3_list), 4)
    w4_avage_norm = np.round(np.mean(w4_list), 4)
    # Export weight
    w = [w1_avage_norm, w2_avage_norm, w3_avage_norm, w4_avage_norm]
    normalized_weight_list.append(w)
    #--------------------------------------------------------------------#
    # Testing models
    test_pred_lgb_mf1024   = my_lgb_mf1024.fit(np.concatenate([train_mf1024, val_mf1024]), np.concatenate([train_label, val_label])).predict_proba(test_mf1024)[::,1]
    test_pred_xgb_mol2vec   = my_xgb_mol2vec.fit(np.concatenate([train_mol2vec, val_mol2vec]), np.concatenate([train_label, val_label])).predict_proba(test_mol2vec)[::,1]
    test_pred_et_rd        = my_et_rd.fit(np.concatenate([train_rd, val_rd]), np.concatenate([train_label, val_label])).predict_proba(test_rd)[::,1]
    test_pred_cat_padel  = my_cat_padel.fit(np.concatenate([train_padel, val_padel]), np.concatenate([train_label, val_label])).predict_proba(test_padel)[::,1]
    test_pred_top4ensemble = (test_pred_lgb_mf1024*w1_avage_norm + test_pred_xgb_mol2vec*w2_avage_norm + test_pred_et_rd*w3_avage_norm + test_pred_cat_padel*w4_avage_norm)
    #--------------------------------------------------------------------#
    # Calculate AUC of all models
    test_roc_auc_lgb_mf1024   = roc_auc_score(test_label, test_pred_lgb_mf1024)
    test_roc_auc_xgb_mol2vec   = roc_auc_score(test_label, test_pred_xgb_mol2vec)
    test_roc_auc_et_rd        = roc_auc_score(test_label, test_pred_et_rd)
    test_roc_auc_cat_padel  = roc_auc_score(test_label, test_pred_cat_padel)
    test_roc_auc_top4ensemble = roc_auc_score(test_label, test_pred_top4ensemble)
    print("meta_AUC: {}".format(np.round(test_roc_auc_top4ensemble, 4)))
    lgb_mf1024_test_auc_list.append(test_roc_auc_lgb_mf1024)
    xgb_mol2vec_test_auc_list.append(test_roc_auc_xgb_mol2vec)
    et_rd_test_auc_list.append(test_roc_auc_et_rd)
    cat_padel_test_auc_list.append(test_roc_auc_cat_padel)
    meta_test_auc_list.append(test_roc_auc_top4ensemble)
    #--------------------------------------------#
    # Export predicted probabilities of all models
    meta_pred_list.append(test_pred_top4ensemble)
    lgb_mf1024_pred_list.append(test_pred_lgb_mf1024)
    xgb_mol2vec_pred_list.append(test_pred_xgb_mol2vec)
    et_rd_pred_list.append(test_pred_et_rd)
    cat_padel_pred_list.append(test_pred_cat_padel)

#============================================================
mean = round(sts.mean(meta_test_auc_list), 4)
sd   = round(sts.stdev(meta_test_auc_list), 4)
ci95 = ( round((mean - 1.96*sd/math.sqrt(10)), 4), round((mean + 1.96*sd/math.sqrt(10)),4))

[LightGBM] [Warning] bagging_fraction is set=0.4, subsample=1.0 will be ignored. Current value: bagging_fraction=0.4
[LightGBM] [Warning] feature_fraction is set=0.8316682559759652, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8316682559759652
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
0:	learn: 0.6394802	total: 22.4ms	remaining: 6.7s
1:	learn: 0.6049499	total: 39.8ms	remaining: 5.93s
2:	learn: 0.5728840	total: 56.3ms	remaining: 5.57s
3:	learn: 0.5469974	total: 73.6ms	remaining: 5.45s
4:	learn: 0.5194281	total: 90.9ms	remaining: 5.36s
5:	learn: 0.4985535	total: 107ms	remaining: 5.26s
6:	learn: 0.4776553	total: 126ms	remaining: 5.28s
7:	learn: 0.4580354	total: 144ms	remaining: 5.24s
8:	learn: 0.4396836	total: 161ms	remaining: 5.21s
9:	learn: 0.4251633	total: 178ms	remaining: 5.16s
10:	learn: 0.4082561	total: 197ms	remaining: 5.17s
11:	learn: 0.3946026	total: 216ms	remaining: 5.18s
12:	learn: 0.38

162:	learn: 0.0991620	total: 2.94s	remaining: 2.47s
163:	learn: 0.0982378	total: 2.96s	remaining: 2.45s
164:	learn: 0.0970125	total: 2.98s	remaining: 2.44s
165:	learn: 0.0963894	total: 3s	remaining: 2.42s
166:	learn: 0.0951590	total: 3.01s	remaining: 2.4s
167:	learn: 0.0946736	total: 3.03s	remaining: 2.38s
168:	learn: 0.0940296	total: 3.05s	remaining: 2.36s
169:	learn: 0.0934680	total: 3.07s	remaining: 2.35s
170:	learn: 0.0924729	total: 3.09s	remaining: 2.33s
171:	learn: 0.0915662	total: 3.1s	remaining: 2.31s
172:	learn: 0.0908765	total: 3.12s	remaining: 2.29s
173:	learn: 0.0902863	total: 3.14s	remaining: 2.27s
174:	learn: 0.0897923	total: 3.16s	remaining: 2.25s
175:	learn: 0.0892775	total: 3.17s	remaining: 2.24s
176:	learn: 0.0884856	total: 3.19s	remaining: 2.22s
177:	learn: 0.0877518	total: 3.21s	remaining: 2.2s
178:	learn: 0.0871228	total: 3.23s	remaining: 2.18s
179:	learn: 0.0868919	total: 3.24s	remaining: 2.16s
180:	learn: 0.0858440	total: 3.26s	remaining: 2.14s
181:	learn: 0.0852

22:	learn: 0.3166479	total: 440ms	remaining: 5.3s
23:	learn: 0.3128848	total: 459ms	remaining: 5.28s
24:	learn: 0.3090159	total: 477ms	remaining: 5.25s
25:	learn: 0.3047888	total: 495ms	remaining: 5.21s
26:	learn: 0.2986715	total: 512ms	remaining: 5.18s
27:	learn: 0.2947402	total: 529ms	remaining: 5.14s
28:	learn: 0.2900439	total: 548ms	remaining: 5.12s
29:	learn: 0.2862015	total: 568ms	remaining: 5.11s
30:	learn: 0.2812946	total: 588ms	remaining: 5.11s
31:	learn: 0.2773404	total: 606ms	remaining: 5.07s
32:	learn: 0.2737442	total: 625ms	remaining: 5.05s
33:	learn: 0.2685852	total: 643ms	remaining: 5.03s
34:	learn: 0.2660910	total: 662ms	remaining: 5.01s
35:	learn: 0.2644674	total: 681ms	remaining: 4.99s
36:	learn: 0.2614244	total: 699ms	remaining: 4.96s
37:	learn: 0.2580981	total: 717ms	remaining: 4.94s
38:	learn: 0.2562980	total: 736ms	remaining: 4.92s
39:	learn: 0.2527538	total: 754ms	remaining: 4.9s
40:	learn: 0.2488791	total: 773ms	remaining: 4.88s
41:	learn: 0.2472044	total: 791ms

184:	learn: 0.0959589	total: 3.36s	remaining: 2.09s
185:	learn: 0.0953572	total: 3.38s	remaining: 2.07s
186:	learn: 0.0943634	total: 3.4s	remaining: 2.06s
187:	learn: 0.0938043	total: 3.42s	remaining: 2.04s
188:	learn: 0.0934874	total: 3.44s	remaining: 2.02s
189:	learn: 0.0934160	total: 3.46s	remaining: 2s
190:	learn: 0.0930116	total: 3.48s	remaining: 1.99s
191:	learn: 0.0924913	total: 3.49s	remaining: 1.97s
192:	learn: 0.0918841	total: 3.51s	remaining: 1.95s
193:	learn: 0.0910472	total: 3.53s	remaining: 1.93s
194:	learn: 0.0909458	total: 3.55s	remaining: 1.91s
195:	learn: 0.0903406	total: 3.57s	remaining: 1.89s
196:	learn: 0.0896283	total: 3.59s	remaining: 1.88s
197:	learn: 0.0894553	total: 3.6s	remaining: 1.86s
198:	learn: 0.0888844	total: 3.62s	remaining: 1.84s
199:	learn: 0.0881932	total: 3.64s	remaining: 1.82s
200:	learn: 0.0874499	total: 3.66s	remaining: 1.8s
201:	learn: 0.0868658	total: 3.68s	remaining: 1.78s
202:	learn: 0.0865034	total: 3.7s	remaining: 1.77s
203:	learn: 0.08642

35:	learn: 0.2644674	total: 659ms	remaining: 4.83s
36:	learn: 0.2614244	total: 677ms	remaining: 4.81s
37:	learn: 0.2580981	total: 699ms	remaining: 4.82s
38:	learn: 0.2562980	total: 716ms	remaining: 4.79s
39:	learn: 0.2527538	total: 736ms	remaining: 4.78s
40:	learn: 0.2488791	total: 756ms	remaining: 4.77s
41:	learn: 0.2472044	total: 774ms	remaining: 4.76s
42:	learn: 0.2454160	total: 792ms	remaining: 4.73s
43:	learn: 0.2432084	total: 809ms	remaining: 4.71s
44:	learn: 0.2409847	total: 828ms	remaining: 4.69s
45:	learn: 0.2386899	total: 846ms	remaining: 4.67s
46:	learn: 0.2372098	total: 865ms	remaining: 4.66s
47:	learn: 0.2352660	total: 885ms	remaining: 4.64s
48:	learn: 0.2336872	total: 904ms	remaining: 4.63s
49:	learn: 0.2316334	total: 922ms	remaining: 4.61s
50:	learn: 0.2302496	total: 942ms	remaining: 4.6s
51:	learn: 0.2289049	total: 960ms	remaining: 4.58s
52:	learn: 0.2272953	total: 979ms	remaining: 4.56s
53:	learn: 0.2257781	total: 999ms	remaining: 4.55s
54:	learn: 0.2244486	total: 1.01

...

160:	learn: 0.1144020	total: 2.94s	remaining: 2.54s
161:	learn: 0.1136878	total: 2.96s	remaining: 2.52s
162:	learn: 0.1120904	total: 2.98s	remaining: 2.5s
163:	learn: 0.1114140	total: 3s	remaining: 2.48s
164:	learn: 0.1107778	total: 3.01s	remaining: 2.46s
165:	learn: 0.1106183	total: 3.03s	remaining: 2.44s
166:	learn: 0.1094065	total: 3.05s	remaining: 2.43s
167:	learn: 0.1083153	total: 3.06s	remaining: 2.41s
168:	learn: 0.1075341	total: 3.08s	remaining: 2.39s
169:	learn: 0.1067577	total: 3.1s	remaining: 2.37s
170:	learn: 0.1052024	total: 3.12s	remaining: 2.35s
171:	learn: 0.1046429	total: 3.14s	remaining: 2.33s
172:	learn: 0.1039401	total: 3.16s	remaining: 2.32s
173:	learn: 0.1038080	total: 3.17s	remaining: 2.3s
174:	learn: 0.1032648	total: 3.19s	remaining: 2.28s
175:	learn: 0.1028409	total: 3.21s	remaining: 2.26s
176:	learn: 0.1012657	total: 3.23s	remaining: 2.24s
177:	learn: 0.1006834	total: 3.24s	remaining: 2.22s
178:	learn: 0.1001048	total: 3.26s	remaining: 2.2s
179:	learn: 0.09932

11:	learn: 0.3985481	total: 225ms	remaining: 5.41s
12:	learn: 0.3875802	total: 244ms	remaining: 5.38s
13:	learn: 0.3794720	total: 263ms	remaining: 5.38s
14:	learn: 0.3680601	total: 283ms	remaining: 5.37s
15:	learn: 0.3609890	total: 302ms	remaining: 5.36s
16:	learn: 0.3531878	total: 321ms	remaining: 5.34s
17:	learn: 0.3456734	total: 339ms	remaining: 5.31s
18:	learn: 0.3384429	total: 358ms	remaining: 5.3s
19:	learn: 0.3324981	total: 379ms	remaining: 5.3s
20:	learn: 0.3263753	total: 398ms	remaining: 5.28s
21:	learn: 0.3219606	total: 416ms	remaining: 5.26s
22:	learn: 0.3166479	total: 434ms	remaining: 5.23s
23:	learn: 0.3128848	total: 453ms	remaining: 5.21s
24:	learn: 0.3090159	total: 472ms	remaining: 5.19s
25:	learn: 0.3047888	total: 490ms	remaining: 5.16s
26:	learn: 0.2986715	total: 509ms	remaining: 5.15s
27:	learn: 0.2947402	total: 529ms	remaining: 5.14s
28:	learn: 0.2900439	total: 549ms	remaining: 5.13s
29:	learn: 0.2862015	total: 567ms	remaining: 5.1s
30:	learn: 0.2812946	total: 585ms	

178:	learn: 0.1001048	total: 3.31s	remaining: 2.24s
179:	learn: 0.0993276	total: 3.33s	remaining: 2.22s
180:	learn: 0.0987996	total: 3.35s	remaining: 2.2s
181:	learn: 0.0973520	total: 3.37s	remaining: 2.18s
182:	learn: 0.0966919	total: 3.38s	remaining: 2.16s
183:	learn: 0.0960477	total: 3.4s	remaining: 2.15s
184:	learn: 0.0959589	total: 3.42s	remaining: 2.13s
185:	learn: 0.0953572	total: 3.44s	remaining: 2.11s
186:	learn: 0.0943634	total: 3.46s	remaining: 2.09s
187:	learn: 0.0938043	total: 3.48s	remaining: 2.07s
188:	learn: 0.0934874	total: 3.49s	remaining: 2.05s
189:	learn: 0.0934160	total: 3.51s	remaining: 2.03s
190:	learn: 0.0930116	total: 3.53s	remaining: 2.01s
191:	learn: 0.0924913	total: 3.55s	remaining: 2s
192:	learn: 0.0918841	total: 3.57s	remaining: 1.98s
193:	learn: 0.0910472	total: 3.59s	remaining: 1.96s
194:	learn: 0.0909458	total: 3.6s	remaining: 1.94s
195:	learn: 0.0903406	total: 3.62s	remaining: 1.92s
196:	learn: 0.0896283	total: 3.64s	remaining: 1.9s
197:	learn: 0.08945

33:	learn: 0.2685852	total: 637ms	remaining: 4.98s
34:	learn: 0.2660910	total: 655ms	remaining: 4.96s
35:	learn: 0.2644674	total: 675ms	remaining: 4.95s
36:	learn: 0.2614244	total: 695ms	remaining: 4.94s
37:	learn: 0.2580981	total: 714ms	remaining: 4.92s
38:	learn: 0.2562980	total: 733ms	remaining: 4.91s
39:	learn: 0.2527538	total: 752ms	remaining: 4.89s
40:	learn: 0.2488791	total: 771ms	remaining: 4.87s
41:	learn: 0.2472044	total: 790ms	remaining: 4.85s
42:	learn: 0.2454160	total: 809ms	remaining: 4.83s
43:	learn: 0.2432084	total: 826ms	remaining: 4.8s
44:	learn: 0.2409847	total: 844ms	remaining: 4.78s
45:	learn: 0.2386899	total: 862ms	remaining: 4.76s
46:	learn: 0.2372098	total: 881ms	remaining: 4.74s
47:	learn: 0.2352660	total: 900ms	remaining: 4.72s
48:	learn: 0.2336872	total: 918ms	remaining: 4.7s
49:	learn: 0.2316334	total: 935ms	remaining: 4.67s
50:	learn: 0.2302496	total: 954ms	remaining: 4.66s
51:	learn: 0.2289049	total: 974ms	remaining: 4.65s
52:	learn: 0.2272953	total: 992ms

203:	learn: 0.0864295	total: 3.77s	remaining: 1.77s
204:	learn: 0.0859449	total: 3.79s	remaining: 1.76s
205:	learn: 0.0855600	total: 3.81s	remaining: 1.74s
206:	learn: 0.0849138	total: 3.83s	remaining: 1.72s
207:	learn: 0.0843605	total: 3.85s	remaining: 1.7s
208:	learn: 0.0838132	total: 3.87s	remaining: 1.68s
209:	learn: 0.0834979	total: 3.89s	remaining: 1.67s
210:	learn: 0.0825935	total: 3.91s	remaining: 1.65s
211:	learn: 0.0816780	total: 3.93s	remaining: 1.63s
212:	learn: 0.0811138	total: 3.94s	remaining: 1.61s
213:	learn: 0.0805355	total: 3.96s	remaining: 1.59s
214:	learn: 0.0797263	total: 3.98s	remaining: 1.57s
215:	learn: 0.0791948	total: 4s	remaining: 1.55s
216:	learn: 0.0784793	total: 4.01s	remaining: 1.53s
217:	learn: 0.0778163	total: 4.03s	remaining: 1.52s
218:	learn: 0.0773042	total: 4.05s	remaining: 1.5s
219:	learn: 0.0772142	total: 4.07s	remaining: 1.48s
220:	learn: 0.0768547	total: 4.09s	remaining: 1.46s
221:	learn: 0.0764444	total: 4.11s	remaining: 1.44s
222:	learn: 0.075

In [ ]:
# Export results
trials = []
for i in range(1,11):
    trial = 'trial_' + str(i)
    trials.append(trial)   

outpath = "pso/allseeds/"
if os.path.isdir(outpath) is False:
    os.makedirs(outpath)
    
w_df = pd.DataFrame(normalized_weight_list).T
w_df.columns = trials
w_df.to_csv("weight_allseeds.csv", index=False)

combined_auc = pd.DataFrame([lgb_mf1024_test_auc_list, xgb_mol2vec_test_auc_list, cat_rd_test_auc_list, et_padel_test_auc_list, meta_test_auc_list]).T
combined_auc.columns = ['lgb_mf1024', 'xgb_mol2vec', 'cat_rd_pred', 'et_padel', 'meta-classifier']
combined_auc.to_csv("PSO_auc_allseeds.csv", index=False)

meta_pred         = pd.DataFrame(meta_pred_list).T
meta_pred.columns = trials
meta_pred.to_csv("PSO_pred_meta_top4_allseeds.csv", index=False)

lgb_mf1024_pred         = pd.DataFrame(lgb_mf1024_pred_list).T
lgb_mf1024_pred.columns = trials
lgb_mf1024_pred .to_csv("PSO_pred_lgb_mf1024_allseeds.csv", index=False)

xgb_mol2vec_pred         = pd.DataFrame(xgb_mol2vec_pred_list).T
xgb_mol2vec_pred.columns = trials
xgb_mol2vec_pred.to_csv("PSO_pred_xgb_mol2vec_allseeds.csv", index=False)

cat_rd_pred         = pd.DataFrame(cat_rd_pred_list).T
cat_rd_pred.columns = trials
cat_rd_pred.to_csv("PSO_pred_et_rd_allseeds.csv", index=False)

et_padel_pred         = pd.DataFrame(et_padel_pred_list).T
et_padel_pred.columns = trials
et_padel_pred.to_csv("PSO_pred_cat_padel_allseeds.csv", index=False)

test_label   = pd.DataFrame(pd.read_csv('label_test.csv'), columns=['true_label'])
error_ana_df = pd.concat([lgb_mf1024_pred.iloc[:,0], xgb_mol2vec_pred.iloc[:,0], cat_rd_pred.iloc[:,0], et_padel_pred.iloc[:,0], meta_pred.iloc[:,0], test_label], axis=1)
error_ana_df.columns = ['lgb_mf1024', 'xgb_mol2vec', 'et_rd', 'cat_padel', 'meta', 'true_label']
error_ana_df.to_csv("error_analysis.csv", index=False)

In [ ]:
sd